THIS FILE CONTAINS THE FEATURE ENGINEERING AND TIMESTAMP ALIGNMENT OF THE TRAIN DATA

                                       Importing the necessary file

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from tqdm import tqdm

In [ ]:
file=open('/content/drive/MyDrive/Project Energy Consumption/df_train_merge_cleaned_imputed.txt','rb')
df_train_merge_cleaned_imputed=pickle.load(file)

In [ ]:
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
from sklearn.model_selection import StratifiedKFold,KFold
import lightgbm as lgb

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from scipy.stats import stats
import datetime as dt

                                  **Function to reduce the memory usage**

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

**AIR TEMPERATURE ALIGNMENT**

1.   HERE THE AIR TEMPERATURE IS NOT ALIGNED WITH THE LOCAL TIMESTAMP OF THE METER READINGS.

2.   FOR 13 DIFFERENT SITES THE AIR TEMPERATURE MAXIMIZES AROUND 19:00 PM TO 23:00 PM WHICH NEEDS TO BE CORRECTED SO THAT THE MODEL CAN BEST IDENTIFY THE ENERGY PATTERN ACCORDING TO THE TEMPERATURE VARIATIONS OVER THE HOUR.



In [ ]:
from datetime import timedelta

In [ ]:
df_train_site_0=df_train_merge_cleaned_imputed[df_train_merge_cleaned_imputed['site_id']==0]
df_train_site_0.reset_index(inplace=True)
df_train_site_0['timestamp_aligned']=df_train_site_0['timestamp']-timedelta(hours=5,minutes=0)
df_air_temp_timestamp=df_train_site_0[['timestamp_aligned','building_id','meter','air_temperature']].copy()
df_air_temp_timestamp.rename(columns={'timestamp_aligned':'timestamp'},inplace=True)
df_train_site_0.drop(['air_temperature','timestamp_aligned'],axis=1,inplace=True)
df_train_site_0['air_temperature_aligned']=df_air_temp_timestamp[df_air_temp_timestamp['timestamp'].isin(df_train_site_0['timestamp'])].reset_index(drop=True)['air_temperature']
df_train_site_0['air_temperature_aligned']=df_train_site_0['air_temperature_aligned'].interpolate()
df_train_site_0.rename(columns={'air_temperature_aligned':'air_temperature'},inplace=True)
df_train_site_0.drop(['level_0','index'],axis=1,inplace=True)

In [ ]:
df_train_site_1=df_train_merge_cleaned_imputed[df_train_merge_cleaned_imputed['site_id']==1]
df_train_site_1.reset_index(inplace=True)
df_train_site_1.drop(['index','level_0'],axis=1,inplace=True)

In [ ]:
df_train_site_2=df_train_merge_cleaned_imputed[df_train_merge_cleaned_imputed['site_id']==2]
df_train_site_2.reset_index(inplace=True)
df_train_site_2['timestamp_aligned']=df_train_site_2['timestamp']-timedelta(hours=7,minutes=0)
df_air_temp_timestamp=df_train_site_2[['timestamp_aligned','building_id','meter','air_temperature']].copy()
df_air_temp_timestamp.rename(columns={'timestamp_aligned':'timestamp'},inplace=True)
df_train_site_2.drop(['air_temperature','timestamp_aligned'],axis=1,inplace=True)
df_train_site_2['air_temperature_aligned']=df_air_temp_timestamp[df_air_temp_timestamp['timestamp'].isin(df_train_site_2['timestamp'])].reset_index(drop=True)['air_temperature']
df_train_site_2['air_temperature_aligned']=df_train_site_2['air_temperature_aligned'].interpolate()
df_train_site_2.rename(columns={'air_temperature_aligned':'air_temperature'},inplace=True)
df_train_site_2.drop(['level_0','index'],axis=1,inplace=True)

In [ ]:
df_train_site_3=df_train_merge_cleaned_imputed[df_train_merge_cleaned_imputed['site_id']==3]
df_train_site_3.reset_index(inplace=True)
df_train_site_3['timestamp_aligned']=df_train_site_3['timestamp']-timedelta(hours=5,minutes=0)
df_air_temp_timestamp=df_train_site_3[['timestamp_aligned','building_id','meter','air_temperature']].copy()
df_air_temp_timestamp.rename(columns={'timestamp_aligned':'timestamp'},inplace=True)
df_train_site_3.drop(['air_temperature','timestamp_aligned'],axis=1,inplace=True)
df_train_site_3['air_temperature_aligned']=df_air_temp_timestamp[df_air_temp_timestamp['timestamp'].isin(df_train_site_3['timestamp'])].reset_index(drop=True)['air_temperature']
df_train_site_3['air_temperature_aligned']=df_train_site_3['air_temperature_aligned'].interpolate()
df_train_site_3.rename(columns={'air_temperature_aligned':'air_temperature'},inplace=True)
df_train_site_3.drop(['level_0','index'],axis=1,inplace=True)

In [ ]:
df_train_site_4=df_train_merge_cleaned_imputed[df_train_merge_cleaned_imputed['site_id']==4]
df_train_site_4.reset_index(inplace=True)
df_train_site_4['timestamp_aligned']=df_train_site_4['timestamp']-timedelta(hours=8,minutes=0)
df_air_temp_timestamp=df_train_site_4[['timestamp_aligned','building_id','meter','air_temperature']].copy()
df_air_temp_timestamp.rename(columns={'timestamp_aligned':'timestamp'},inplace=True)
df_train_site_4.drop(['air_temperature','timestamp_aligned'],axis=1,inplace=True)
df_train_site_4['air_temperature_aligned']=df_air_temp_timestamp[df_air_temp_timestamp['timestamp'].isin(df_train_site_4['timestamp'])].reset_index(drop=True)['air_temperature']
df_train_site_4['air_temperature_aligned']=df_train_site_4['air_temperature_aligned'].interpolate()
df_train_site_4.rename(columns={'air_temperature_aligned':'air_temperature'},inplace=True)
df_train_site_4.drop(['level_0','index'],axis=1,inplace=True)

In [ ]:
df_train_site_5=df_train_merge_cleaned_imputed[df_train_merge_cleaned_imputed['site_id']==5]
df_train_site_5.reset_index(inplace=True)
df_train_site_5.drop(['index','level_0'],axis=1,inplace=True)

In [ ]:
df_train_site_6=df_train_merge_cleaned_imputed[df_train_merge_cleaned_imputed['site_id']==6]
df_train_site_6.reset_index(inplace=True)
df_train_site_6['timestamp_aligned']=df_train_site_6['timestamp']-timedelta(hours=5,minutes=0)
df_air_temp_timestamp=df_train_site_6[['timestamp_aligned','building_id','meter','air_temperature']].copy()
df_air_temp_timestamp.rename(columns={'timestamp_aligned':'timestamp'},inplace=True)
df_train_site_6.drop(['air_temperature','timestamp_aligned'],axis=1,inplace=True)
df_train_site_6['air_temperature_aligned']=df_air_temp_timestamp[df_air_temp_timestamp['timestamp'].isin(df_train_site_6['timestamp'])].reset_index(drop=True)['air_temperature']
df_train_site_6['air_temperature_aligned']=df_train_site_6['air_temperature_aligned'].interpolate()
df_train_site_6.rename(columns={'air_temperature_aligned':'air_temperature'},inplace=True)
df_train_site_6.drop(['level_0','index'],axis=1,inplace=True)

In [ ]:
df_train_site_7=df_train_merge_cleaned_imputed[df_train_merge_cleaned_imputed['site_id']==7]
df_train_site_7.reset_index(inplace=True)
df_train_site_7['timestamp_aligned']=df_train_site_7['timestamp']-timedelta(hours=5,minutes=0)
df_air_temp_timestamp=df_train_site_7[['timestamp_aligned','building_id','meter','air_temperature']].copy()
df_air_temp_timestamp.rename(columns={'timestamp_aligned':'timestamp'},inplace=True)
df_train_site_7.drop(['air_temperature','timestamp_aligned'],axis=1,inplace=True)
df_train_site_7['air_temperature_aligned']=df_air_temp_timestamp[df_air_temp_timestamp['timestamp'].isin(df_train_site_7['timestamp'])].reset_index(drop=True)['air_temperature']
df_train_site_7['air_temperature_aligned']=df_train_site_7['air_temperature_aligned'].interpolate()
df_train_site_7.rename(columns={'air_temperature_aligned':'air_temperature'},inplace=True)
df_train_site_7.drop(['level_0','index'],axis=1,inplace=True)

In [ ]:
df_train_site_8=df_train_merge_cleaned_imputed[df_train_merge_cleaned_imputed['site_id']==8]
df_train_site_8.reset_index(inplace=True)
df_train_site_8['timestamp_aligned']=df_train_site_8['timestamp']-timedelta(hours=5,minutes=0)
df_air_temp_timestamp=df_train_site_8[['timestamp_aligned','building_id','meter','air_temperature']].copy()
df_air_temp_timestamp.rename(columns={'timestamp_aligned':'timestamp'},inplace=True)
df_train_site_8.drop(['air_temperature','timestamp_aligned'],axis=1,inplace=True)
df_train_site_8['air_temperature_aligned']=df_air_temp_timestamp[df_air_temp_timestamp['timestamp'].isin(df_train_site_8['timestamp'])].reset_index(drop=True)['air_temperature']
df_train_site_8['air_temperature_aligned']=df_train_site_8['air_temperature_aligned'].interpolate()
df_train_site_8.rename(columns={'air_temperature_aligned':'air_temperature'},inplace=True)
df_train_site_8.drop(['level_0','index'],axis=1,inplace=True)

In [ ]:
df_train_site_9=df_train_merge_cleaned_imputed[df_train_merge_cleaned_imputed['site_id']==9]
df_train_site_9.reset_index(inplace=True)
df_train_site_9['timestamp_aligned']=df_train_site_9['timestamp']-timedelta(hours=6,minutes=0)
df_air_temp_timestamp=df_train_site_9[['timestamp_aligned','building_id','meter','air_temperature']].copy()
df_air_temp_timestamp.rename(columns={'timestamp_aligned':'timestamp'},inplace=True)
df_train_site_9.drop(['air_temperature','timestamp_aligned'],axis=1,inplace=True)
df_train_site_9['air_temperature_aligned']=df_air_temp_timestamp[df_air_temp_timestamp['timestamp'].isin(df_train_site_9['timestamp'])].reset_index(drop=True)['air_temperature']
df_train_site_9['air_temperature_aligned']=df_train_site_9['air_temperature_aligned'].interpolate()
df_train_site_9.rename(columns={'air_temperature_aligned':'air_temperature'},inplace=True)
df_train_site_9.drop(['level_0','index'],axis=1,inplace=True)

In [ ]:
df_train_site_10=df_train_merge_cleaned_imputed[df_train_merge_cleaned_imputed['site_id']==10]
df_train_site_10.reset_index(inplace=True)
df_train_site_10['timestamp_aligned']=df_train_site_10['timestamp']-timedelta(hours=7,minutes=0)
df_air_temp_timestamp=df_train_site_10[['timestamp_aligned','building_id','meter','air_temperature']].copy()
df_air_temp_timestamp.rename(columns={'timestamp_aligned':'timestamp'},inplace=True)
df_train_site_10.drop(['air_temperature','timestamp_aligned'],axis=1,inplace=True)
df_train_site_10['air_temperature_aligned']=df_air_temp_timestamp[df_air_temp_timestamp['timestamp'].isin(df_train_site_10['timestamp'])].reset_index(drop=True)['air_temperature']
df_train_site_10['air_temperature_aligned']=df_train_site_10['air_temperature_aligned'].interpolate()
df_train_site_10.rename(columns={'air_temperature_aligned':'air_temperature'},inplace=True)
df_train_site_10.drop(['level_0','index'],axis=1,inplace=True)

In [ ]:
df_train_site_11=df_train_merge_cleaned_imputed[df_train_merge_cleaned_imputed['site_id']==11]
df_train_site_11.reset_index(inplace=True)
df_train_site_11['timestamp_aligned']=df_train_site_11['timestamp']-timedelta(hours=5,minutes=0)
df_air_temp_timestamp=df_train_site_11[['timestamp_aligned','building_id','meter','air_temperature']].copy()
df_air_temp_timestamp.rename(columns={'timestamp_aligned':'timestamp'},inplace=True)
df_train_site_11.drop(['air_temperature','timestamp_aligned'],axis=1,inplace=True)
df_train_site_11['air_temperature_aligned']=df_air_temp_timestamp[df_air_temp_timestamp['timestamp'].isin(df_train_site_11['timestamp'])].reset_index(drop=True)['air_temperature']
df_train_site_11['air_temperature_aligned']=df_train_site_11['air_temperature_aligned'].interpolate()
df_train_site_11.rename(columns={'air_temperature_aligned':'air_temperature'},inplace=True)
df_train_site_11.drop(['level_0','index'],axis=1,inplace=True)

In [ ]:
df_train_site_12=df_train_merge_cleaned_imputed[df_train_merge_cleaned_imputed['site_id']==12]
df_train_site_12.reset_index(inplace=True)
df_train_site_12.drop(['index','level_0'],axis=1,inplace=True)

In [ ]:
df_train_site_13=df_train_merge_cleaned_imputed[df_train_merge_cleaned_imputed['site_id']==13]
df_train_site_13.reset_index(inplace=True)
df_train_site_13['timestamp_aligned']=df_train_site_13['timestamp']-timedelta(hours=6,minutes=0)
df_air_temp_timestamp=df_train_site_13[['timestamp_aligned','building_id','meter','air_temperature']].copy()
df_air_temp_timestamp.rename(columns={'timestamp_aligned':'timestamp'},inplace=True)
df_train_site_13.drop(['air_temperature','timestamp_aligned'],axis=1,inplace=True)
df_train_site_13['air_temperature_aligned']=df_air_temp_timestamp[df_air_temp_timestamp['timestamp'].isin(df_train_site_13['timestamp'])].reset_index(drop=True)['air_temperature']
df_train_site_13['air_temperature_aligned']=df_train_site_13['air_temperature_aligned'].interpolate()
df_train_site_13.rename(columns={'air_temperature_aligned':'air_temperature'},inplace=True)
df_train_site_13.drop(['level_0','index'],axis=1,inplace=True)

In [ ]:
df_train_site_14=df_train_merge_cleaned_imputed[df_train_merge_cleaned_imputed['site_id']==14]
df_train_site_14.reset_index(inplace=True)
df_train_site_14['timestamp_aligned']=df_train_site_14['timestamp']-timedelta(hours=5,minutes=0)
df_air_temp_timestamp=df_train_site_14[['timestamp_aligned','building_id','meter','air_temperature']].copy()
df_air_temp_timestamp.rename(columns={'timestamp_aligned':'timestamp'},inplace=True)
df_train_site_14.drop(['air_temperature','timestamp_aligned'],axis=1,inplace=True)
df_train_site_14['air_temperature_aligned']=df_air_temp_timestamp[df_air_temp_timestamp['timestamp'].isin(df_train_site_14['timestamp'])].reset_index(drop=True)['air_temperature']
df_train_site_14['air_temperature_aligned']=df_train_site_14['air_temperature_aligned'].interpolate()
df_train_site_14.rename(columns={'air_temperature_aligned':'air_temperature'},inplace=True)
df_train_site_14.drop(['level_0','index'],axis=1,inplace=True)

In [ ]:
df_train_site_15=df_train_merge_cleaned_imputed[df_train_merge_cleaned_imputed['site_id']==15]
df_train_site_15.reset_index(inplace=True)
df_train_site_15['timestamp_aligned']=df_train_site_15['timestamp']-timedelta(hours=5,minutes=0)
df_air_temp_timestamp=df_train_site_15[['timestamp_aligned','building_id','meter','air_temperature']].copy()
df_air_temp_timestamp.rename(columns={'timestamp_aligned':'timestamp'},inplace=True)
df_train_site_15.drop(['air_temperature','timestamp_aligned'],axis=1,inplace=True)
df_train_site_15['air_temperature_aligned']=df_air_temp_timestamp[df_air_temp_timestamp['timestamp'].isin(df_train_site_15['timestamp'])].reset_index(drop=True)['air_temperature']
df_train_site_15['air_temperature_aligned']=df_train_site_15['air_temperature_aligned'].interpolate()
df_train_site_15.rename(columns={'air_temperature_aligned':'air_temperature'},inplace=True)
df_train_site_15.drop(['level_0','index'],axis=1,inplace=True)

**END OF AIR TEMPERATURE ALIGNMENT**

**Merging all the sites and then sorting it so that the order of timestamp remains does not changes.**

In [ ]:
df_train_merged_final=pd.concat([df_train_site_0,df_train_site_1,df_train_site_2,df_train_site_3,df_train_site_4,
                                 df_train_site_5,df_train_site_6,df_train_site_7,df_train_site_8,df_train_site_9,
                                 df_train_site_10,df_train_site_11,df_train_site_12,df_train_site_13,df_train_site_14,
                                 df_train_site_15],axis=0)

In [ ]:
df_train_merged_final=df_train_merged_final.sort_values(['timestamp','building_id'])

                                         **Starting with Feature Engineering**

**ENGINEERED FEATURES**

1.   RELATIVE HUMIDITY-->HUMIDITY AFFECTS THE ENERGY REQUIREMENTS OF THE BUILDINGS.AS HUMIDITY INCREASES THE LOAD ON THE HVAC SYSTEM INCREASES DUE TO WHICH THE METER READINGS ARE AFFECTED.THIS FEATURE WILL HELP THE MODEL TO LEARN THE ENRGY REQUIREMENTS ACCORDING TO THE HUMIDITY FACTOR.

2.  IS SUMMER MONTH & IS WINTER MONTH--> CHILLED WATER HOT WATER AND STEAM READINGS SHOW CONSIDERABLE PATTERN OVER THE SEASONS.THIS FEATURE WILL HELP THE MODEL TO LEARN THAT.

3.  HOLIDAYS-->AS THE ENERGY REQUIREMENTS MIGHT BE DIFFERENT DURING THE HOLIDAY AS COMPARED TO THE OTHER DAYS INCLUDING THIS FEATURE WILL HELP THE MODEL TO LEARN THAT.

4.   BUSY-HOURS-->AS THE ENERGY REQUIREMENTS ARE DIFFERENT OVER THE DAY THAN THE NIGHT.SO TO HELP THE MODEL LEARN THAT WE CAN USE BUSY HOURS(6:00 AM TO 18:00 PM). 


5.  IS WEEKDAY-->THIS FEATURE IS ADDED TO CHECK THE ENRGY REQUIREMENTS OF THE WEEKDAY AS COMPARED TO THE WEEKEND.AS OBSERVED IN THE EDA IT SHOWS A SIGNIFICANT TREND OVER THE WEEK AS COMPARED TO THE WEEKEND FOR ALL THE ENERGY METER.


6.   ADDING BASIC TIMESTAMP FEATURES SUCH AS DAY MONTH AND HOUR.








In [ ]:
saturated_vapor_pressure = 6.11 * (10**(7.5*df_train_merged_final['air_temperature']/(237.3+df_train_merged_final['air_temperature'])))
actual_vapor_pressure = 6.11 * (10**(7.5*df_train_merged_final['dew_temperature']/(237.3+df_train_merged_final['dew_temperature'])))
df_train_merged_final['relative_humidity']=(actual_vapor_pressure/saturated_vapor_pressure)*100

In [ ]:
df_train_merged_final['is_winter_month']=(df_train_merged_final['month'].isin([12,1,2])).astype(int)
df_train_merged_final['is_summer_month']=(df_train_merged_final['month'].isin([6,7,8])).astype(int)
df_train_merged_final['weekday']=df_train_merged_final['timestamp'].dt.weekday

In [ ]:
holidays = ["2016-01-01", "2016-01-18", "2016-02-15", "2016-05-30", "2016-07-04",
            "2016-09-05", "2016-10-10", "2016-11-11", "2016-11-24", "2016-12-25"]

In [ ]:
holiday_datetime=pd.to_datetime(holidays,yearfirst=True)
df_train_merged_final['is_pub_holiday']=(df_train_merged_final['timestamp'].dt.date.isin(holiday_datetime.date)).astype(int)
df_train_merged_final['is_weekday']=((~df_train_merged_final['timestamp'].dt.date.isin(holiday_datetime.date))&(df_train_merged_final['weekday'].isin([0,1,2,3,4]))).astype(int)

In [ ]:
z_busy_hours=df_train_merged_final.set_index(['timestamp']).between_time('06:00:00','18:00:00').reset_index()
z_busy_hours_timestamp=[i for i in z_busy_hours['timestamp']]
df_train_merged_final['busy_hours']=((~df_train_merged_final['timestamp'].dt.date.isin(holiday_datetime.date))&(df_train_merged_final['timestamp'].isin(z_busy_hours_timestamp))).astype(int)

In [ ]:
df_train_merged_final['hour']=df_train_merged_final['timestamp'].dt.hour

                                                **End with Feature Engineering**

In [ ]:
df_train_merged_final.reset_index(inplace=True)

In [ ]:
df_train_merged_final_red=reduce_mem_usage(df_train_merged_final,verbose=True)

Mem. usage decreased to 1113.10 Mb (47.3% reduction)


In [ ]:
del df_train_merge_cleaned_imputed
del df_train_site_0,df_train_site_1,df_train_site_2,df_train_site_3,df_train_site_4,df_train_site_5,df_train_site_6,df_train_site_7,df_train_site_8,df_train_site_9,df_train_site_10,df_train_site_11,df_train_site_12,df_train_site_13,df_train_site_14,df_train_site_15

**Doing Label Encoding of the categorical variables**

In [ ]:
label_encoder=LabelEncoder()
df_train_merged_final_red['primary_use']=label_encoder.fit_transform(df_train_merged_final_red['primary_use'])

In [ ]:
df_tr_red_final=reduce_mem_usage(df_train_merged_final_red,verbose=True)

Mem. usage decreased to 981.04 Mb (11.9% reduction)


**Storing the final trained file which is cleaned imputed aligned and feature engineered.**

In [ ]:
filename='df_tr_red_final_modified.txt'
my_file=open(filename,'wb')

In [ ]:
pickle.dump(df_tr_red_final,my_file)
my_file.close()

In [ ]:
#REF-->https://towardsdatascience.com/forecasting-energy-consumption-using-neural-networks-xgboost-2032b6e6f7e2
    #--->https://www.kaggle.com/frednavruzov/aligning-temperature-timestamp
    #--->https://www.weather.gov/media/epz/wxcalc/vaporPressure.pdf
    #--->https://www.kaggle.com/gemartin/load-data-reduce-memory-usage